In [1]:
#import graphviz 
#from IPython.display import Image
#import pydotplus


import pandas as pd #dataframe handling library
import numpy as np #math library

import os.path #path library


# Use bokeh to plot Interactive Plots on separate tab
from bokeh.models.annotations import Title
from bokeh.embed import components, file_html
from bokeh.resources import CDN
from bokeh.io import output_file, show
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.palettes import Reds256, Category10_4,Spectral6
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
#from mpl_toolkits import mplot3d



from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold
from sklearn.linear_model import LinearRegression

from scipy.optimize import fsolve
from scipy.signal import find_peaks #science library peak finder algorithm


import itertools
np.set_printoptions(precision=3, suppress=True)  # suppress scientific float notation

# SETTINGS FOR NOTEBOOK
#To show all columns
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)
pd.set_option('max_colwidth', None)


import pickle
import joblib
#from sklearn.externals import joblib
from joblib import Memory

#rom ipywidgets import interactive

In [2]:
# Check Versions 

import sys 
for name, module in sorted(sys.modules.items()): 
  if hasattr(module, '__version__'): 
    print(name, module.__version__) 

IPython 7.13.0
IPython.core.release 7.13.0
PIL 7.1.2
PIL.Image 7.1.2
PIL._version 7.1.2
_csv 1.0
_ctypes 1.1.0
_decimal 1.70
argparse 1.1
backcall 0.1.0
bokeh 2.0.2
bokeh.document.document 2.0.2
bokeh.resources 2.0.2
bokeh.util.version 2.0.2
bottleneck 1.3.2
cffi 1.14.0
colorama 0.4.3
csv 1.0
ctypes 1.1.0
cycler 0.10.0
dateutil 2.8.1
decimal 1.70
decorator 4.4.2
distutils 3.6.10
immutables 0.11
ipaddress 1.0
ipykernel 5.1.4
ipykernel._version 5.1.4
ipython_genutils 0.2.0
ipython_genutils._version 0.2.0
jedi 0.15.2
jinja2 2.10
joblib 0.11
json 2.0.9
jupyter_client 6.1.3
jupyter_client._version 6.1.3
jupyter_core 4.6.3
jupyter_core.version 4.6.3
kiwisolver 1.2.0
logging 0.5.1.2
markupsafe 1.1.1
matplotlib 3.1.3
matplotlib.backends.backend_agg 3.1.3
mkl 2.3.0
numpy 1.18.1
numpy.core 1.18.1
numpy.core._multiarray_umath 3.1
numpy.lib 1.18.1
numpy.linalg._umath_linalg b'0.1.5'
optparse 1.5.3
pandas 1.0.1
parso 0.5.2
pickleshare 0.7.5
platform 1.0.8
prompt_toolkit 3.0.4
pygments 2.6.1
pyparsi

In [3]:
class Welding_model:
  
  def __init__(self):
    #  Initialiaze Parameters
    #  Replace with read statement for implementation
    
    self.fixed_params = {
      'Nm_prominence':1,   
      'forward_torque_points':30,
      
      'min_gap': 5,
      'max_gap': 20,
      'end_loc': 50,
      'look_forward_start': 400,
      'look_forward_finish': 600,
      'ow_ll': 0.4,
      'ow_ul': 1.4,
      'iw_ll': 1.2,      
      'iw_ul': 2.1
      }   
    
    self.model_params = {
      'force_start': [370,540],
      'mm_iw_offset': [0.1022,0.1022],
      'mm_ow_offset': [-0.0166,-0.0166],
      'force_prominence': [1,1]
      }
    
  def display_params(self):
    print(self.model_params)
    print(self.fixed_params)   
   
    
  def predict(self, Spindle_Position,Spindle_Force,Spindle_Torque, head, sample_num, start_time, end_time):
    '''
      Main function to compute Inner and Outer Welt
      
      Spindle_Position(mm)   Array
      Spindle_Force(N)       Array
      Spindle_TorqueNm)      Array
      head                   int
      sample_num             int
      start_time             time
      end_time               time 
      
    '''
  
    error_msg="Error --  at Initialize"
    try:
    
      # Create DataFrame from 3 input streams
      # Add the sample number and a column to track the peaks that are identified
      df = pd.DataFrame({'mm': Spindle_Position, 'N': Spindle_Force, 'Nm':Spindle_Torque }) 
      df['mm_avg']=df['mm'].rolling(center=True, window=10, min_periods=1).mean() 
      df['Nm_avg']=df['Nm'].rolling(center=True, window=5, min_periods=1).mean() 
      df['N_avg']=df['N'].rolling(center=True, window=5, min_periods=1).mean() 
      df['points1']=0      # for Force 
      df['points2']=0      # for Torque
      df['points3']=0      # other
      
      df['sample']= sample_num
      
      error_msg="Error --  After Initialize"
       
      if len(Spindle_Force) < 1000:
        error_msg="Error --  # of points is too low :  " + str(len(Spindle_Force))
        a=1/0
      error_msg="Error --  After Initialize 2"
      
      # --- IW Estimate ---
      tsi=df[df['Nm_avg']>=.05].index[0]                                       # initial point in Torque
      df.loc[df.index==tsi , 'points2']=1
      iws=df.loc[tsi,'mm_avg']                                                 # equivalent starting point in Position (shift to correct for Bias)
      
      error_msg="Error --  After Initialize 3"
      # --- Alternative method for IW estimate
      fst=df[df['N'] > self.model_params['force_start'][head-1]].index[0]
      df.loc[df.index==fst , 'points1']=1                                                           # force start position
      
      error_msg="Error --  After Initialize 4"
      #Find peaks in Torque and Force curves
      pks_torque=find_peaks(df['Nm_avg'],prominence= self.fixed_params['Nm_prominence'])[0]
      pks_force=find_peaks(df['N_avg'],prominence = self.model_params['force_prominence'][head-1])[0]
      error_msg='Error --  Peaks Identified Torque: ' + str(len(pks_torque)) + '\tForce: ' + str(len(pks_force))
      
      error_msg="Error --  After Initialize 5"
      max_torque=0
      max_point=0     
      for u in range(len(pks_torque)): 
          df.loc[df.index==pks_torque[u] , 'points2']=1   #tag the points for labeling
          if df.loc[df.index==pks_torque[u] , 'Nm_avg'].values[0] > max_torque:
            max_point=u
            max_torque=df.loc[df.index==pks_torque[u] , 'Nm_avg'].values[0]
      pt=pks_torque[max_point]
      pt2=pks_torque[max_point+1]
      #print(pks_torque[max_point],pt,pt2)
      
      
      # Find the max Force after initial identified condition in Torqur curve
      max_force=0
      max_point_force=0     
      ow_pk_index=[]
      for u in range(len(pks_force)):
          if pks_force[u]>=pt + self.fixed_params['forward_torque_points'] and pks_force[u]<=pt2:
            ow_pk_index.append(pks_force[u])
            if df.loc[df.index==pks_force[u] , 'N_avg'].values[0] > max_force:
              max_point_force= pks_force[u]
              max_force= df.loc[df.index==pks_force[u] , 'N_avg'].values[0]
      #print('max force  ',max_point,max_force)
      #print(ow_pk_index)
      error_msg="Error --  After Initialize 6"
      
      
      # Use the second peak in the Torque curve to identify the final Position (in mm curve)
      final_mm=df.loc[(pks_torque[1]+ self.fixed_params['look_forward_start']):(pks_torque[1]+ self.fixed_params['look_forward_finish'])]['mm_avg'].median()
      #final_mm=df['mm_avg'].max()
      
      iwp=final_mm - iws
      error_msg="Error --  After IW estimate :" + str(iwp)
      
      # Identify Peak with largest drop in Force .. ad the last point to the pks_force ARRAY
      #pks_force = np.append(pks_force , 0)
      pks_force=np.sort(ow_pk_index)
      
      prev_N=0
      max_drop=-1
      max_drop_pos=0
      for u in range(len(ow_pk_index)):
          if ow_pk_index[u] >= max_point_force:                                   #search only in points past the largest pea
            df.loc[df.index==ow_pk_index[u] , 'points1']=1
            drop=df.loc[df.index==ow_pk_index[u] , 'N_avg'].values[0] - prev_N
            #print('drop: ', drop,'\tpoint and N value: ',ow_pk_index[u], df.loc[df.index==ow_pk_index[u] , 'N'].values[0])
            if drop<max_drop:
                max_drop=drop
                max_drop_pos=ow_pk_index[u-1]    #take the previous point
            prev_N=df.loc[df.index==ow_pk_index[u] , 'N_avg'].values[0]
            
      if max_drop_pos==0:
        max_drop_pos=ow_pk_index[-1]
      error_msg='Error --  Peaks Identified Torque: ' + str(len(pks_torque)) + '\tForce Kept: ' + str(len(ow_pk_index))
      #print('max drop poit: ', max_drop_pos)
      
      # Estimate OW
      ows=df.loc[max_drop_pos,'mm_avg']                                                # use the last peak before the largest drop in the N curve
      owp=final_mm-ows
      
      
      # apply Bias correction
      iwp = iwp +  self.model_params['mm_iw_offset'][head-1]
      owp = owp +  self.model_params['mm_ow_offset'][head-1]
      #print(iwp,owp)
                                                            
                                                                
      df.loc[df.index==pt2+self.fixed_params['look_forward_start'] , 'points2']=1                   # end position
      df.loc[df.index==pt+self.fixed_params['forward_torque_points'] , 'points2']=1                 # shifted start
      df.loc[max_drop_pos , 'points3']=1                                               
                                                                 

      error_msg='Error --  during Tagging data'
    
      # --- Set IW, OW and Overall Flags -- if either IW or OW fail, the overall sample will fail also
      sample_flag= 'PASS'
      iw_flag='PASS'
      ow_flag='PASS'
    
      if (iwp<self.fixed_params['iw_ll']):
        iw_flag='LOW'
        sample_flag= 'FAIL'
      elif (iwp>self.fixed_params['iw_ul']):
        iw_flag='HIGH'
        sample_flag= 'FAIL'

      if (owp<self.fixed_params['ow_ll']):
        ow_flag='LOW'
        sample_flag= 'FAIL'
      elif (owp>self.fixed_params['ow_ul']):
        ow_flag='HIGH'
        sample_flag= 'FAIL'
        
    
      # Check to ensure the data that came into the function was not truncated 
      if df.iloc[0]['N'] > self.model_params['force_start'][head-1] + 300:
        error_msg = 'WARNING - Initial Force too high ' + str(df.iloc[0]['N'])
      else:
        error_msg = 'ok'
        
      return np.round(iwp,4),np.round(owp,4),iw_flag, ow_flag, sample_flag,self.fixed_params['iw_ll'], self.fixed_params['iw_ul'], self.fixed_params['ow_ll'], self.fixed_params['ow_ul'], \
              sample_num,start_time,end_time, head, df, error_msg
    
    except:
      if error_msg.find('kept')>0:           #tag the points it found before checking for gap
        for u in range(len(pks_force)-1):  
           df.loc[df.index==pks_force[u] , 'points1']=1
        
      return 0.0, 0.0, 'ERROR','ERROR','ERROR',self.fixed_params['iw_ll'], self.fixed_params['iw_ul'], self.fixed_params['ow_ll'], self.fixed_params['ow_ul'], \
              sample_num,start_time,end_time, head, df,error_msg 



In [4]:
iw_ow = Welding_model()

iw_ow.display_params()

{'force_start': [370, 540], 'mm_iw_offset': [0.1022, 0.1022], 'mm_ow_offset': [-0.0166, -0.0166], 'force_prominence': [1, 1]}
{'Nm_prominence': 1, 'forward_torque_points': 30, 'min_gap': 5, 'max_gap': 20, 'end_loc': 50, 'look_forward_start': 400, 'look_forward_finish': 600, 'ow_ll': 0.4, 'ow_ul': 1.4, 'iw_ll': 1.2, 'iw_ul': 2.1}


In [8]:
dir_path = 'C:/Users/rosen.o/OneDrive - Procter and Gamble/Smart Platforms/Welding/Files to Azure/'
weld_data_file='Physical Weld Data.csv'

lab_results = pd.read_csv(dir_path + '/' + weld_data_file, header= 0)

lab_results.head()

,Weld_ID,Timestamp,Head,IW Avg (mm),OW Avg (mm),Sample ID,file
0,10001,7/14/2021 1:07:21,1,1.35675,0.36675,1,210513 Physical Weld Data.xlsx
1,10002,7/14/2021 1:07:21,1,1.33600,0.30100,2,210513 Physical Weld Data.xlsx
2,10003,7/14/2021 1:07:21,1,1.32425,0.31050,3,210513 Physical Weld Data.xlsx
3,10004,7/14/2021 1:07:21,1,1.42125,0.41400,4,210513 Physical Weld Data.xlsx
4,10005,7/14/2021 1:07:21,1,1.35200,0.35400,5,210513 Physical Weld Data.xlsx


In [9]:
detail_data_file='combined_2.csv'

detail_data = pd.read_csv(dir_path + '/' + detail_data_file, header= 0)

detail_data.head()

,time,Spindle_Pos,Spindle_Force,Spindle_Torque,sample,head
0,0.0,0.0094,0.5905,0.0011,10001,1.0
1,1.0,0.0149,0.5830,-0.0042,10001,1.0
2,2.0,0.0170,0.6560,-0.0033,10001,1.0
3,3.0,0.0133,0.6164,0.0011,10001,1.0
4,4.0,0.0091,0.6237,-0.0036,10001,1.0


In [18]:
# predict(self, Spindle_Position,Spindle_Force,Spindle_Torque, head, sample_num, start_time, end_time):

sample = 10098
selected_point = detail_data.loc[detail_data['sample'] == sample]
selected_point.reset_index(inplace=True)

hd=lab_results.loc[lab_results['Weld_ID'] == sample,'Head'].values[0]
ts=lab_results.loc[lab_results['Weld_ID'] == sample,'Timestamp'].values[0]
print(hd, ts)


selected_point.head()

2 7/14/2021  1:07:21


,index,time,Spindle_Pos,Spindle_Force,Spindle_Torque,sample,head
0,648188,0.0,-5.6425,-2.7605,0.0027,10098,2.0
1,648189,1.0,-5.6422,-2.7966,-0.0006,10098,2.0
2,648190,2.0,-5.6429,-2.7321,-0.0009,10098,2.0
3,648191,3.0,-5.6421,-2.7111,-0.0009,10098,2.0
4,648192,4.0,-5.6419,-2.8189,-0.0034,10098,2.0


In [23]:
est = iw_ow.predict(selected_point['Spindle_Pos'] ,selected_point['Spindle_Force'],selected_point['Spindle_Torque'] ,hd , sample, ts, ts)

print(est[0:13], est[14])
print(lab_results.loc[lab_results.Weld_ID==sample,['IW Avg (mm)','OW Avg (mm)']])

(1.5123, 0.8717, 'PASS', 'PASS', 'PASS', 1.2, 2.1, 0.4, 1.4, 10098, '7/14/2021  1:07:21', '7/14/2021  1:07:21', 2) ok
    IW Avg (mm)  OW Avg (mm)
96      1.46875      0.81225


In [25]:
if est[14]!='aa':
  df_out = est[13]
  df_out.loc[df_out.points1 == 0 , "points1"] = np.nan
  df_out.loc[df_out.points2 == 0 , "points2"] = np.nan
  df_out.loc[df_out.points3 == 0 , "points3"] = np.nan
  
  df_out['N_points']= df_out['N'] * df_out['points1'] 
  df_out['N_points_2']= df_out['N'] * df_out['points2']
  df_out['N_points_3']= df_out['N'] * df_out['points3']
  
  df_out['Nm_points']= df_out['Nm'] * df_out['points1'] 
  df_out['Nm_points_2']= df_out['Nm'] * df_out['points2'] 
  df_out['Nm_points_3']= df_out['Nm'] * df_out['points3']
  
  df_out['mm_points']= df_out['mm_avg'] * df_out['points1'] 
  df_out['mm_points_2']= df_out['mm_avg'] * df_out['points2'] 
  df_out['mm_points_3']= df_out['mm_avg'] * df_out['points3']
  
  #What should have been the position based on the lab result
  end_mm= df_out['mm_points_2'].max()
  #print(end_mm)
  
  iw_actual=lab_results.loc[lab_results['Weld_ID']==sample,'IW Avg (mm)'].values[0]
  ow_actual=lab_results.loc[lab_results['Weld_ID']==sample,'OW Avg (mm)'].values[0]
  iwsp=end_mm-iw_actual
  owsp=end_mm-ow_actual
  iwi=df_out[df_out['mm_avg']>=iwsp].index[0]
  owi=df_out[df_out['mm_avg']>=owsp].index[0]

  df_out['points4']=float("nan")
  df_out.loc[df_out.index==iwi , 'points4']=1
  df_out.loc[df_out.index==owi , 'points4']=1
  
  df_out['N_points_4']= df_out['N'] * df_out['points4']
  df_out['mm_points_4']= df_out['mm_avg'] * df_out['points4']
  df_out['Nm_points_4']= df_out['Nm'] * df_out['points4']

  start_f= df_out['N_points_4'].min()
  print(start_f)


948.9718


In [ ]:

p1 = figure(title="Nm", x_axis_label='time', y_axis_label='Nm')
p1.line(df.index, df['Nm'], line_width=2, color="blue", legend_label="Nm")
p1.line(df.index, df['Nm_avg'], line_width=2, color="green", line_dash='dotted', legend_label="Nm_avg")
p1.circle(df.index, df['Nm_points'], size=5, color="red")
p1.circle(df.index, df['Nm_points_actual'], size=7, color="green")
p1.legend.location = "bottom_left"
t = Title()
t.text = "Sample # " + str(sample) + "    Nm"
p1.title=t
p1.legend.click_policy="hide"

p2 = figure(title="mm", x_axis_label='time', y_axis_label='mm')
p2.line(df.index, df['mm'], line_width=2, color="blue", legend_label="mm")
p2.line(df.index, df['mm_avg'], line_width=2, color="green", line_dash='dotted', legend_label="mm_avg")
p2.circle(df.index, df['mm_points'], size=5, color="red")
p2.circle(df.index, df['mm_points_actual'], size=7, color="green")
p2.legend.location = "bottom_left"
p2.legend.click_policy="hide"

p3 = figure(title="N", x_axis_label='time', y_axis_label='N')
p3.line(df.index, df['N'], line_width=2, color="blue", legend_label="N")
p3.line(df.index, df['N_avg'], line_width=2, color="green", line_dash='dotted', legend_label="N_avg")
p3.circle(df.index, df['N_points'], size=5, color="red")
p3.circle(df.index, df['N_points_actual'], size=7, color="green")
p3.legend.location = "bottom_left"
p3.legend.click_policy="hide"


grid = gridplot([[p1,p2,p3]], plot_width=500, plot_height=500)
show(grid)